## Missing: 

- Boundary condition for dust particle! 
     - start with a square, work up if time allows

In [542]:
import math
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt 
import sys, os 
import time
import csv
%matplotlib inline

In [543]:
# constants 
EPS0 = 8.85 * 10 ** -12
QE = 1.602 * 10 ** -19 
K = 1.381 * 10 ** -23 
AMU = 1.661 * 10 ** -27
MI = 40 * AMU #mass of an Ar+ ion 

#simulation inputs 
N = 10 #max particles 
n0 = 1 * 10 ** 12 #density
V_ref = 0 #reference potential 
Te = 1  #electron temperature in eV 
Ti = 0.1 #ion temperature in eV
V_d = -5.1 #dust potential (use values from lit)
v_drift = 7000
tol = 1 * 10 ** -4
iterations = 10


#plasma parameters 
lambd_D = np.sqrt( EPS0 * Te /(n0 * QE) )
vth = np.sqrt((2 * QE * Ti) / MI)
v0 = 0.2 #stream velocity 


#domain parameters 
nx = 100
ny = 100
J = nx * ny 
dx = dy = dh = lambd_D
Lx = nx * dx 
Ly = ny * dy 
dt = (0.1 * dx) / v_drift 
np_in = (J-1) * 15 

#specific weight 
flux = n0 * v_drift * Ly 
npt = flux * dt #particles created per timestep 
sw = npt / np_in #specific weight 
q_mp = 1 #macroparticle charge 

# particle arrays
p_pos = np.zeros([N,2])
p_velo = np.zeros([N,2])

print(len(p_pos), type(p_pos), sw, q_mp)

10 <class 'numpy.ndarray'> 3683.264706 1


In [544]:
def charge_depo(dh, NP, J):

    q = np.zeros((J, J))
    rho = np.zeros((J, J))

    for p in range (1, NP): 
            
            fi = (1 + p_pos[p,0]) / (dh) 
            i = np.floor(fi)
            hx = fi - i  
                
            fj = (1 + p_pos[p,1]) / (dh)
            j = np.floor(fj)
            hy = fj - j 
                        
            q[i,j] = q[i,j] + (1-hx) * (1-hy)
            q[i+1, j] = q[i+1, j] + hx * (1-hy)
            q[i, j+1] = q[i, j+1] + (1-hx) * hy 
            q[i+1, j+1] = q[i+1, j+1] + hx * hy 
            
            rho = (sw + q_mp * q) / (dh * dh)
                
            #rho[0,:] = 2 * rho[0,:]
            #rho[-1, :] = 2 * rho[-1, :]
            #rho[:, 0] = 2 * rho[:, 0]
            #rho[:, -1] = 2 * rho[:, -1]
            
            rho = rho + (1 * 10 ** 4)    
    return q, rho 

In [545]:
#stencil array used in potential solver 

def stencil_fd(J, dx, dh, nx, ny):
    """Uses the finite difference scheme"""
    #E = []
    
    #source = rho
    M = np.zeros((J,J))
    
    for i in range (1, nx-1):
        for j in range(1, ny-1): 
            u = (j-1) * nx + i
            M[u, u] = -4. / (dh * dh) 
            M[u, u-1] = 1. / (dh * dh)
            M[u, u+1] = 1. / (dh * dh)
            M[u, u-nx] = 1. / (dh * dh)
            M[u, u+ (nx-2)] = 1. / (dh * dh)    
    
    for i in range (0, nx):
        u = i+1
        M[u,u] = -1. / dh
        M[u,u+nx] = 1. / dh 
        
    for i in range (0,nx): 
        u = (ny-1) * nx  + i 
        M[u, u-nx] = 1. / dh
        M[u, u] = -1. / dh 
        
    for j in range (0, ny): 
        u = (j-1) * nx + nx 
        M[u, :] = np.zeros([1, J])
        M[u, u-1] = 1. / dh 
        M[u, u] = -1. / dh 
        
    for j in range(0, ny): 
        u = (j-1) * nx+1 
        M[u, :] = np.zeros([1, J])
        M[u, u] = 1.     
        
    return M

In [546]:
def solver_2d(M, rho, tol, Ti, n0, V_ref, QE): 
    'Uses Gauss Sidel method'
    
    iterations = 2000 
    
    M = stencil_fd(J, dx, dh, nx, ny)
    V = np.ones((nx,ny)) * V_ref
    
    source = rho
    NX = NY = np.size(source,1)
    NN = np.size(source)
    
    b0 = np.reshape(source, (NN,1))
    x = np.reshape(V, (np.size(V), 1))
    
    #print(np.size(x), np.size(b0))
    
    b = b0 - n0 * np.exp((x-V_ref)/ Ti)
    b = b * QE / EPS0     
    
    for it in range (0, iterations):
        
        b[0:NX] = 0 
        b[NN-NX+1 : NN] = 0 
        b[NX: NX : NN] = 0 
        b[1:NX:NN] = V_ref
        
        if it % 10 == 0:
            R = np.linalg.norm(b-M*x)
            
            if R <= tol:
                print('Solver converged!')
            
    if R > tol: 
        print('Solver failed to converge. Try tweaking your inputs.')
    
    V = np.reshape(x, (NX,NY))
    
    return V
   

In [548]:
#Main cycle 

NP = 0

print('Calculating...')

while NP <= N:  
    
    rho = charge_depo(dh, NP, J)
    print(rho, np.size(rho))
    
    NP  = NP + 1 
    #E = E_from_V(rho, J, dx, dh, nx, ny)
    
    #potential solver 
    V = solver_2d(M, rho, tol, Ti, n0, V_ref, QE)
    
    
    #E field solver
    Ex = np.zeros([nx, ny])
    Ey = np.zeros([nx, ny])
    E = np.zeros([nx, ny])
    
    Ex[1:nx-1, :] = V[0: nx - 3, :] - V[2 : nx-1, :] 
    Ey[1:ny-1, :] = V[0: ny - 3, :] - V[2 : ny-1, :] 
    
    #forward difference on... 
    Ex[0, :] = V[0, :] - V[1, :] #x=0
    Ex[-1, :] = V[-2, :] - V[-1, :] #x=Lx 
    Ey[:, 0] = V[:, 0] - V[:, 1] #y=0
    Ey[:, -1] = V[:, -2] - V[:, -1] #y=Ly
    
    Ex = Ex / (2 * dx)                                        
    Ey = Ey / (2 * dy)
    
    
    #generate particles 
    
    if NP + np_in >= N:
        np_in = N - NP
   
    #insert paritcles 
    p_pos[N+1:N+np_in,0] = np.random(np_in,1) * dh #x position 
    p_pos[N+1:N+np_in,1] = np.random(np_in,1) * Ly #y position 
    
    #sample Maxwellian in x,y 
    #add drift velocity in x 
    p_velo[N+1:N+np_in,0] = v_drift + (-1.5 + np.random(np_in,1) 
                                     + np.random(np_in,1) + np.random(np_in, 1)) * vth 
    p_velo[N+1:N+np_in,1] = 0.5 * (-1.5 + rand(np_in,1) 
                            + np.random(np_in,1) + np.random(np_in, 1)) * vth
    
    #move particles 
    
    p = 1 
    
    while p <= NP:
            
        fi = 1 + p_pos[p,0]/dx 
        i = np.floor(fi)
        hx = fi - i  
        
        fj = 1+ p_pos[p,1]/dy 
        j = np.floor(fj)
        hy = fj - j 
            
        E = ([Ex[i, j], Ey[i,j]]) * (1-hx) * (1-hy)                                                
        E = E + ([Ex[i + 1, j], Ey[i + 1, j]]) * hx * (1-hy)                                                
        E = E + ([Ex[i, j + 1], Ey[i + 1, j]]) * (1-hx) * hy                                                
        E = E + ([Ex[i + 1, j + 1], Ey[i + 1, j + 1]]) * hx * hy  
            
        F = QE * E 
        a = F/MI 
        
        p_pos[p, :] = p_velo[p, :] + a * dt 
        p_velo[p, :] = p_pos[p, :] + p_velo[p, :] * dt 
        
        if p_pos[p,1] < 0: 
            p_pos[p,1] = -p_pos[p,1]
            p_velo[p,1] = -p_velo[p,1]
            
        p = p + 1 
#

Calculating...
(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])) 200000000


/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: RuntimeWarning: overflow encountered in exp


ValueError: operands could not be broadcast together with shapes (200000000,1) (100000000,1) 